In [ ]:
from os.path import basename, join, splitext
import itertools
import matplotlib.pyplot as plt
import numpy as np
from numpy import ceil, log10
import pandas as pd
import seaborn as sb
from subprocess import call
import warnings
%matplotlib inline
%run util.ipynb
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)

In [ ]:
class PlotGenerator:
    pt_list = [2,3,8,4,6,12,1,10,14]
    
    def __init__(self, data_info, theta, ymax_offset):
        self.data_info = data_info
        self.theta = theta
        
#         self.goal = None
        self.xlabel = None
#         self.ylabel = None
#         self.xrange = None
        self.ylabel = 'Avg. exe. time \(sec\)'
        self.xtics = None
        self.xmax = 0
        self.ymax = 0
        self.ymax_offset = ymax_offset
        self.xlog = True
        self.xformat = "10^{%L}"
        
        self.title_list = []
        self.setting_list = setting_list
   
    def execute(data_info, theta, ymax_offset=0):
        if data_info.size == '*': TimeBySizePlotGenerator(data_info, theta, ymax_offset).run()
        elif data_info.qlen == '*': TimeByQlenPlotGenerator(data_info, theta, ymax_offset).run()
        elif data_info.nr == '*': TimeByNrulePlotGenerator(data_info, theta, ymax_offset).run()
        elif data_info.lr == '*': TimeByLenRatioPlotGenerator(data_info, theta, ymax_offset).run()

    def run(self):
        self.output_name = '{}__{}__{:.1f}'.format(self.goal, self.data_info.name, self.theta).replace('.', '_')
        self.load_df()
        self.write_result()
        self.output_script()
        self.output_plot()
        
    def load_df(self):
        pass
        
    def parse_row(self, row):
        pass

    def update_xmax(self, row):
        pass
    
    def update_ymax(self, row):
        self.ymax = max(self.ymax, row.Time_SearchPerQuery_MEAN)
    
    def additional_settings(self, f):
        pass
    
    def write_result(self):
        self.path_result = join('result', '{}.result'.format(self.output_name))
        df = self.df
        with open(self.path_result, 'w') as f:
            for setting in self.setting_list:
                f.write("# {}\n".format(setting))
                df_target = df[df.theta==self.theta][df.setting==setting]
                for row in df_target.itertuples():
                    self.update_xmax(row)
                    self.update_ymax(row)
                    f.write(self.parse_row(row))
                f.write('\n\n')
                self.title_list.append(setting if df_target.size > 0 else None)

    
    def output_script(self):
        self.path_script = join('script', '{}.plot'.format(self.output_name))
        self.path_plot = join('plot', '{}.pdf'.format(self.output_name))
        with open(self.path_script, 'w') as f:
#             f.write('set terminal font \",20\"\n')
            if self.xlabel is not None: f.write('set xlabel \"{}\" font \",20\"\n'.format(self.xlabel))
            f.write('set ylabel \"{}\" font \",20\"\n'.format(self.ylabel))
            if self.xtics is not None: f.write('set xtics {}\n'.format(self.xtics))
            f.write('set xtics font \",16"\n')
            f.write('set ytics font \",16\"\n')
            f.write('set xrange {}\n'.format(self.xrange))
#             f.write('set yrange [*:{}]\n'.format(10**(ceil(log10(self.ymax))+self.ymax_offset)))
            f.write('set yrange [*:{}]\n'.format(10**(4.5+self.ymax_offset)))
            f.write('set key vertical maxrows 5\n')
            if self.xlog:
                f.write('set logscale x\n')
                f.write('set format x \"{}\"\n'.format(self.xformat))
            f.write('set logscale y\n')
            f.write('set format y \"10^{%L}\"\n')
            f.write('set size 0.6,0.5\n')
            self.additional_settings(f)
            

            f.write('set term postscript\n')
            f.write('set output\"| ps2pdf - {}\"\n'.format(self.path_plot))

            cmd_list = []
            idx = 0
            for tidx, title in enumerate(self.title_list):
                if title is None: continue
                cmd = '\"{}\" index {} with linespoints lc \"black\" lw 2 ps 1.5 dt {} pt {} title \"{}\"'.format(
                self.path_result, idx, tidx+1, PlotGenerator.pt_list[tidx%len(PlotGenerator.pt_list)], title)
                cmd_list.append(cmd)
                idx += 1
            f.write('plot\\\n'+',\\\n'.join(cmd_list))    #',\\\n'.join(cmd_list)
    
    def output_plot(self):
        call(['gnuplot', self.path_script])
        call(['sleep', '0.3'])
        call(['pdfcrop', self.path_plot, 'tmp_plot'])
        call(['mv', 'tmp_plot', self.path_plot])

In [ ]:
class TimeBySizePlotGenerator(PlotGenerator):

    def __init__(self, data_info, theta, ymax_offset, alg_list=alg_list, setting_list=setting_list, goal=None):
        super(TimeBySizePlotGenerator, self).__init__(data_info, theta, ymax_offset)
        self.goal = 'time_by_size' if goal is None else goal
        
        if self.data_info.name.endswith('-DOC'):
#             self.xlabel = 'Number of documents'
            self.xrange = '[1000:1000000]'
        else:
#             self.xlabel = 'Number of strings'
            self.xrange = '[10000:10000000]'
              
        self.xlog = True
        self.ymax_offset = ymax_offset
        self.alg_list = alg_list
        self.setting_list = setting_list

    def load_df(self):
        self.df = df_time_by_size(self.data_info, self.alg_list)
#         if self.data_info.name in ['WIKI', 'AMAZON']:
#             self.df.drop( self.df[self.df['n']==10000000].index , inplace=True)
#             self.xrange = f"[{min(self.df['n'])}:{max(self.df['n'])}]"
    
    def parse_row(self, row):
        if self.data_info.name.endswith('-DOC'):
            return '{}\t{}\n'.format(row.n_doc, row.Time_SearchPerQuery_MEAN)
        else:
            return '{}\t{}\n'.format(row.n, row.Time_SearchPerQuery_MEAN)

In [ ]:
class TimeByQlenPlotGenerator(PlotGenerator):

    def __init__(self, data_info, theta, ymax_offset):
        super(TimeByQlenPlotGenerator, self).__init__(data_info, theta, ymax_offset)
        self.goal = 'time_by_qlen'
#         self.xlabel = 'Query size
        self.xrange = '[1:9]'
        self.xlog = False
        self.ymax_offset = ymax_offset

    def load_df(self):
        self.df = df_time_by_qlen(self.data_info)
    
    def parse_row(self, row):
        return '{}\t{}\n'.format(row.qlen, row.Time_SearchPerQuery_MEAN)
    
    def additional_settings(self, f):
        f.write('set xtics 1,2,9\n')

In [ ]:
class TimeByNrulePlotGenerator(PlotGenerator):

    def __init__(self, data_info, theta, ymax_offset):
        super(TimeByNrulePlotGenerator, self).__init__(data_info, theta, ymax_offset)
        self.goal = 'time_by_nr'
#         self.xlabel = 'Number of rules'
        self.xrange = '[1000:110000]'
        self.xlog = True
        self.xformat = "2{/Symbol\\264}10^{%L}"
        
    def load_df(self):
        self.df = df_time_by_nr(self.data_info)
    
    def parse_row(self, row):
        return '{}\t{}\n'.format(row.nr, row.Time_SearchPerQuery_MEAN)

In [ ]:
class TimeByLenRatioPlotGenerator(PlotGenerator):

    def __init__(self, data_info, theta, ymax_offset):
        super(TimeByLenRatioPlotGenerator, self).__init__(data_info, theta, ymax_offset)
        self.goal = 'time_by_lr'
#         self.xlabel = 'Ratio of string size'
        self.xrange = '[0.2:1.0]'
        self.xtics = 0.2
        self.xlog = False
        self.xformat = "%.1f"
        
    def load_df(self):
        self.df = df_time_by_lr(self.data_info)
    
    def parse_row(self, row):
        return '{}\t{}\n'.format(row.lr, row.Time_SearchPerQuery_MEAN)

In [ ]:
class TimeByThetaPlotGenerator(PlotGenerator):

    def __init__(self, data_info, theta, ymax_offset):
        super(TimeByThetaPlotGenerator, self).__init__(data_info, theta, ymax_offset)
        self.goal = 'time_by_theta'
#         self.xlabel = 'Minimum threshold'
        self.xrange = '[0.6:1.0]'
        self.xlog = False
        
    def load_df(self):
        self.df = df_time_by_filtering(self.data_info).sort_values('theta').groupby(['theta', 'setting']).head(1)
    
    def parse_row(self, row):
        return '{}\t{}\n'.format(row.theta, row.Time_SearchPerQuery_MEAN)
    
    def additional_settings(self, f):
        f.write('set xtics 0.6,0.1,1.0\n')
    
    def write_result(self):
        self.path_result = join('result', '{}.result'.format(self.output_name))
        df = self.df
        with open(self.path_result, 'w') as f:
            for setting in self.setting_list:
                f.write("# {}\n".format(setting))
                df_target = df[df.setting==setting]
                for row in df_target.itertuples():
                    self.update_xmax(row)
                    self.update_ymax(row)
                    f.write(self.parse_row(row))
                f.write('\n\n')
                self.title_list.append(setting if df_target.size > 0 else None)

In [ ]:
def plot_verify_by_alg(data_info, theta):
    goal = 'verify_by_alg'
#     df = df_time_by_filtering(data_info)
    df = df_from_PrefixSearchFilterPowerTest()
    df = df[~pd.isnull(df.setting)]
    df = df[df.data_name==data_info.name][df.n==int(data_info.size)][df.nr==int(data_info.nr)][df.qlen==int(data_info.qlen)]
    output_name = '{}__{}__{:.1f}'.format(goal, data_info.name, theta).replace('.', '_')
    df_target = df[df.theta==theta].sort_values(['setting'])
    title_list = []

    path_result = join('result', '{}.result'.format(output_name))
    if data_info.name == 'WIKI': ymin = 10**2
    elif data_info.name == 'PUBMED': ymin = 10**2
    elif data_info.name == 'AMAZON': ymin = 10**2
    ymax = 10**12
    with open(path_result, 'w') as f:
        for idx, row in enumerate(df_target.itertuples()): 
            try: 
                title = row.setting
                title_list.append(title)
                f.write('{}\t{}\t{}\t{}\t{}\n'.format(idx, title, row.Num_Verified, row.Num_QS_Verified, row.Num_TS_Verified))
#                 ymin = min(ymin, row.Num_Verified)
            except: pass

    path_script = join('script', '{}.plot'.format(output_name))
    path_plot = join('plot', '{}.pdf'.format(output_name))
    with open(path_script, 'w') as f:
        f.write('set style fill solid border\n')
#         f.write('set xlabel font \",20\"\n')
        f.write('set ylabel \"#Verified pairs\" font \",20\"\n')
        f.write('set yrange [{}:{}]\n'.format(ymin, ymax))
        f.write('set xtics font \",16"\n')
        f.write('set ytics font \",16\"\n')
        f.write('set key vertical maxrows 5\n')

        f.write('set logscale y\n')
        f.write('set format y \"10^{%L}\"\n')
    #     f.write('set style fill pattern border -1\n')
        f.write('set boxwidth 0.9\n')
        f.write('set xtics format \"\"\n')
        f.write('set size 0.6,0.6\n')

        f.write('set term postscript\n')
        f.write('set output\"| ps2pdf - {}\"\n'.format(path_plot))

        cmd_list = []
        for idx, title in enumerate(title_list):
            cmd = '\"{}\" every ::{}::{} using 1:3 with boxes fs pattern {} lw 2 title \"{}\"'.format(
            path_result, idx, idx, idx+2, title)
            cmd_list.append(cmd)
        f.write('plot\\\n'+',\\\n'.join(cmd_list))

    call(['gnuplot', path_script])
    call(['sleep', '0.3'])
    call(['pdfcrop', path_plot, 'tmp_plot'])
    call(['mv', 'tmp_plot', path_plot])

In [ ]:
def plot_filter_time_by_alg(data_info, theta):
    goal = 'filter_time_by_alg'
#     df = df_time_by_filtering(data_info)
    df = df_from_PrefixSearchFilterPowerTest()
    df = df[~pd.isnull(df.setting)]
    df = df[df.data_name==data_info.name][df.n==int(data_info.size)][df.nr==int(data_info.nr)][df.qlen==int(data_info.qlen)]
    output_name = '{}__{}__{:.1f}'.format(goal, data_info.name, theta).replace('.', '_')
    df_target = df[df.theta==theta].sort_values(['setting'])
    title_list = []

    path_result = join('result', '{}.result'.format(output_name))
#     if data_info.name == 'WIKI': ymin = 10**3
#     elif data_info.name == 'PUBMED': ymin = 10**3
#     elif data_info.name == 'AMAZON': ymin = 10**3
    ymin = 10**(-1)
    ymax = 10**(5.5)
    with open(path_result, 'w') as f:
        for idx, row in enumerate(df_target.itertuples()): 
            try: 
                title = row.setting
                title_list.append(title)
                f.write('{}\t{}\t{}\n'.format(idx, title, row.Time_Total))
#                 ymin = min(ymin, row.Num_Verified)
            except: pass

    path_script = join('script', '{}.plot'.format(output_name))
    path_plot = join('plot', '{}.pdf'.format(output_name))
    with open(path_script, 'w') as f:
        f.write('set style fill solid border\n')
#         f.write('set xlabel font \",20\"\n')
        f.write('set ylabel \"#Verified pairs\" font \",20\"\n')
        f.write('set yrange [{}:{}]\n'.format(ymin, ymax))
        f.write('set xtics font \",16"\n')
        f.write('set ytics font \",16\"\n')
        f.write('set key vertical maxrows 5\n')

        f.write('set logscale y\n')
        f.write('set format y \"10^{%L}\"\n')
    #     f.write('set style fill pattern border -1\n')
        f.write('set boxwidth 0.9\n')
        f.write('set xtics format \"\"\n')
        f.write('set size 0.6,0.6\n')

        f.write('set term postscript\n')
        f.write('set output\"| ps2pdf - {}\"\n'.format(path_plot))

        cmd_list = []
        for idx, title in enumerate(title_list):
            cmd = '\"{}\" every ::{}::{} using 1:3 with boxes fs pattern {} lw 2 title \"{}\"'.format(
            path_result, idx, idx, idx+2, title)
            cmd_list.append(cmd)
        f.write('plot\\\n'+',\\\n'.join(cmd_list))

    call(['gnuplot', path_script])
    call(['sleep', '0.3'])
    call(['pdfcrop', path_plot, 'tmp_plot'])
    call(['mv', 'tmp_plot', path_plot])

In [ ]:
def plot_index_build_time(varying, theta=0.7):
    goal = 'index_build_time'
    n0 = '1000000'
    q0 = '5'
    lr0 = '1.0'
    output_name = '{}__{}'.format(goal, varying).replace('.', '_')
    title_list = ['WIKI', 'PUBMED', 'AMAZON']
    
    if varying == 'nt': 
        data_info_list = [DataInfo('WIKI', '*', '107836', q0, lr0), DataInfo('PUBMED', '*', '79011', q0, lr0), DataInfo('AMAZON', '*', '107836', q0, lr0),]
        xlabel = '\#strings'
        xrange = '[10000:1000000]'
        xformat = "10^{%L}"
    elif varying == 'nr': 
        data_info_list = [DataInfo('WIKI', n0, '*', q0, lr0), DataInfo('PUBMED', n0, '*', q0, lr0), DataInfo('AMAZON', n0, '*', q0, lr0),]
        xlabel = '\#rules'
        xrange = '[1000:110000]'
        xformat = "2{/Symbol\\264}10^{%L}"
    
    path_result = join('result', '{}.result'.format(output_name))
    with open(path_result, 'w') as f:
        for data_info in data_info_list:
            if varying == 'nt':
                df = df_time_by_size(data_info)
                df = df[df.theta==theta][df.setting=='PF'][['n', 'Time_BuildIndex']]
            elif varying == 'nr':
                df = df_time_by_nr(data_info)
                df = df[df.theta==theta][df.setting=='PF'][['nr', 'Time_BuildIndex']]

            for row in df.itertuples(): 
                try: 
                    f.write('{}\t{}\n'.format(row[1], row[2]))
    #                 ymin = min(ymin, row.Num_Verified)
                except: pass
            f.write('\n\n')

    path_script = join('script', '{}.plot'.format(output_name))
    path_plot = join('plot', '{}.pdf'.format(output_name))
    with open(path_script, 'w') as f:
        f.write('set xlabel \"{}\" font \",20\"\n'.format(xlabel))
        f.write('set ylabel \"{}\" font \",20\"\n'.format("Time \(sec\)"))
        f.write('set xtics font \",16"\n')
        f.write('set ytics font \",16\"\n')
        f.write('set xrange {}\n'.format(xrange))
#         f.write('set yrange [*:{}]\n'.format(10**(ceil(log10(self.ymax))+self.ymax_offset)))
#         f.write('set key vertical maxrows 4\n')
        f.write('set logscale x\n')
        f.write('set format x \"{}\"\n'.format(xformat))
        f.write('set logscale y\n')
        f.write('set format y \"10^{%L}\"\n')
        f.write('set size 0.6,0.6\n')

        f.write('set term postscript\n')
        f.write('set output\"| ps2pdf - {}\"\n'.format(path_plot))

        cmd_list = []
        for idx, title in enumerate(title_list):
            cmd = '\"{}\" index {} with linespoints lc \"black\" lw 2 ps 1.5 dt {} pt {} title \"{}\"'.format(
            path_result, idx, idx+1, PlotGenerator.pt_list[idx%len(PlotGenerator.pt_list)], title_list[idx])
            cmd_list.append(cmd)
        f.write('plot\\\n'+',\\\n'.join(cmd_list))    #',\\\n'.join(cmd_list)

    call(['gnuplot', path_script])
    call(['sleep', '0.3'])
    call(['pdfcrop', path_plot, 'tmp_plot'])
    call(['mv', 'tmp_plot', path_plot])

In [ ]:
def tex_data_stat():
    df = pd.read_csv('DataStat.txt', sep='\t', names=['label', 'n_doc', 'nt', 'n_tokens', 'nr', 'nSnt_min', 'nSnt_max', 'nSnt_mean', 'nSnt_var', 'len_min', 'len_max', 'len_mean', 'len_var', 'na_min', 'na_max', 'na_mean', 'na_var'])

    SEP = ' & '
    EOL = ' \\\\\\hline'
    df0 = df[df.n_doc==0][['nt', 'n_tokens', 'nr', 'len_mean', 'na_mean']]
    row_headers = ["\# of strings", "\# of distinct tokens", "\# of rules", "\makecell{Avg. string size}", "\makecell{Avg. \# of applicable rules}", ]
    for header, row in zip(row_headers, df0.items()):
        val_list = row[1].tolist()
        if type(val_list[0]) == int: print(header+SEP+SEP.join(map(lambda x: f"{x:,}", val_list))+EOL)
        elif type(val_list[0]) == float: print(header+SEP+SEP.join(map(lambda x: f"{x:.3f}", val_list))+EOL)
    print()

    df1 = df[df.n_doc>0][['n_doc', 'nt', 'n_tokens', 'nSnt_mean']]
    row_headers = ["\# of documents", "\# of strings", "\# of distinct tokens", "\makecell{Avg. \# of strings in a document}", ]
    for header, row in zip(row_headers, df1.items()):
        val_list = row[1].tolist()
        if type(val_list[0]) == int: print(header+SEP+SEP.join(map(lambda x: f"{x:,}", val_list))+EOL)
        elif type(val_list[0]) == float: print(header+SEP+SEP.join(map(lambda x: f"{x:.3f}", val_list))+EOL)

In [ ]:
# def tex_heuristic_accuracy():
#     df = df_heuristic_accuracy()
#     for theta in [0.6, 0.7, 0.8, 0.9, 1.0]:
#         line = ''
#         line += '{:.1f}'.format(theta)
#         for data_name in ['WIKI', 'PUBMED', 'AMAZON']:
#             row = df[df.theta==theta][df.data_name==data_name]
#             for side in ['QS', 'TS']:
#                 line += ' & {:.4f}'.format(row['Acc_{}'.format(side)].values[0])
#         print(line+' \\\\\\hline')

In [ ]:
def tex_heuristic_accuracy():
    SEP = ' & '
    EOL = ' \\\\\\hline'
    dataset_list = ['WIKI', 'AMAZON', 'PUBMED']
    outputs = []
    with open('val_acc_result.txt') as f:
        for line in f: outputs.append(dict(map(lambda x:x.split('='), line.strip().split('\t'))))
    df = pd.DataFrame(outputs)
    df['dataset'] = list(map(lambda x:x.split('_')[0], df['dataset'].tolist()))
    df = df.astype({'avgdiff_Q':float, 'avgdiff_T':float, 'nQ1/nQ0':float, 'nT1/nT0':float})
    for theta in ['0.5', '0.6', '0.7', '0.8', '0.9', '1.0']:
        line = theta
        for dataset in dataset_list:
            acc_Q, acc_T = df[df.theta==theta][df.dataset==dataset][['nQ1/nQ0', 'nT1/nT0']].values[0].tolist()
            line += SEP + f"{acc_Q:.4f}" + SEP + f"{acc_T:.4f}"
        line += EOL
        print(line)
    print()
    for theta in ['0.5', '0.6', '0.7', '0.8', '0.9', '1.0']:
        line = theta
        for dataset in dataset_list:
            acc_Q, acc_T = df[df.theta==theta][df.dataset==dataset][['avgdiff_Q', 'avgdiff_T']].values[0].tolist()
            line += SEP + f"{acc_Q:.2e}" + SEP + f"{acc_T:.2e}"
        line += EOL
        print(line)
# tex_heuristic_accuracy()

In [ ]:
def tex_table_filter_power(attr, setting_list=None):
    assert attr in ['Time_SearchPerQuery_MEAN', 'Num_Verified', 'Len_Verified']
    if setting_list is None:
        setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L', 'RSS-R', 'RSS-CL', 'RSS-PL', 'RSS-CP', 'RSS-CR', 'RSS-PR', 'PKDUCK-W', 'RSS-CPL', 'RSS-CPR', 'RSS-CLR', 'RSS-PLR', 'RSS-CPLR']
    datainfo_list = [
            DataInfo('WIKI', '*', '107836', '5', '1.0', '-1'),
#             DataInfo('PUBMED', '*', '79011', '5', '1.0', '-1'),
#             DataInfo('AMAZON', '*', '107836', '5', '1.0', '-1'),
    ]
    df = df_from_PrefixSearchFilterPowerTest(setting_list, path='ZeroPrefixSearch.out.repeat10')

    print((' & '.join(['\\multirow{2}{*}{$\\theta$}']+list(map(lambda x:'\\makecell[c]{\\textit{'+x+'}}', setting_list)))).replace('RSS-', '').replace('PKDUCK-W', 'LR') +' \\\\\\hline')
    for datainfo in datainfo_list:
        df0 = df[df.data_name==datainfo.name][~pd.isnull(df.setting)][df.n==100000][df.nr==int(datainfo.nr)][df.theta>=0.7][df.qlen==int(datainfo.qlen)][[attr, 'Time_TS_searchRecordPF.pkduck', 'setting', 'theta']]

        for theta in [0.7, 0.8, 0.9, 1.0]:
            print('{}'.format(theta), end='')
            for setting in setting_list:
                try:
                    v = df0[df.theta==theta][df.setting==setting][attr].values[0] - df0[df.theta==theta][df.setting==setting]['Time_TS_searchRecordPF.pkduck'].values[0]/100
                    if attr == 'Time_SearchPerQuery_MEAN': print(' & {:,d}'.format(int(v*1000)), end='')
                    elif attr == 'Num_Verified' or attr == 'Len_Verified': print(' & {:,}'.format(v), end='')
                except:
                    print(' & -', end='')
            print(' \\\\\\hline')
tex_table_filter_power('Time_SearchPerQuery_MEAN')
# tex_table_filter_power('Num_Verified')
# tex_table_filter_power('Num_Verified', ['RSS-CP', 'RSS-CPL'])

In [ ]:
def tex_table_filter_power_transposed(attr, setting_list=None):
    assert attr in ['Time_SearchPerQuery_MEAN', 'Num_Verified', 'Len_Verified']
    if setting_list is None:
        setting_list = ['RSS-NAIVE', 'RSS-C', 'RSS-P', 'RSS-L', 'RSS-R', 'RSS-CL', 'RSS-PL', 'RSS-CP', 'RSS-CR', 'RSS-PR', 'PKDUCK-W', 'RSS-CPL', 'RSS-CPR', 'RSS-CLR', 'RSS-PLR', 'RSS-CPLR']
    datainfo_list = [
            DataInfo('WIKI', '*', '107836', '5', '1.0', '-1'),
#             DataInfo('PUBMED', '*', '79011', '5', '1.0', '-1'),
#             DataInfo('AMAZON', '*', '107836', '5', '1.0', '-1'),
    ]
    df = df_from_PrefixSearchFilterPowerTest(setting_list, path='ZeroPrefixSearch.out.repeat10')

#     print((' & '.join(['\\multirow{2}{*}{$\\theta$}']+list(map(lambda x:'\\makecell[c]{\\textit{'+x+'}}', setting_list)))).replace('RSS-', '') +' \\\\\\hline')
    for datainfo in datainfo_list:
        df0 = df[df.data_name==datainfo.name][~pd.isnull(df.setting)][df.n==100000][df.nr==int(datainfo.nr)][df.theta>=0.7][df.qlen==int(datainfo.qlen)][[attr, 'Time_TS_searchRecordPF.pkduck', 'setting', 'theta']]
        for setting in setting_list:
            print(f"\\makecell[c]{{\\textit{{{setting.replace('RSS-', '').replace('PKDUCK-W', 'LR')}}}}}", end='')
            for theta in [0.7, 0.8, 0.9, 1.0]:
                try:
                    v = df0[df.theta==theta][df.setting==setting][attr].values[0] - df0[df.theta==theta][df.setting==setting]['Time_TS_searchRecordPF.pkduck'].values[0]/100
                    if attr == 'Time_SearchPerQuery_MEAN': print(f' & {int(v*1000):,}', end='')
                    elif attr == 'Num_Verified' or attr == 'Len_Verified': print(f' & {v:,}', end='')
                except:
                    print(' & -', end='')
            print(' \\\\\\hline')
#     print(df)
tex_table_filter_power_transposed('Time_SearchPerQuery_MEAN')

In [ ]:
def tex_compare_index(data_info, setting_list=None):
    if setting_list is None: setting_list = ['RSS-C', 'RSS-P', 'PKWISE-R', 'FAERIE-R']
#     datainfo_list = [
#             DataInfo('WIKI', '100000', nr, '5', '1.0'),
#             DataInfo('PUBMED', '100000', nr, '5', '1.0'),
#             DataInfo('AMAZON', '100000', nr, '5', '1.0'),
#             DataInfo('PUBMED', '100000', nr, '5', '1.0'),
#     ]
    theta = 0.7
#     for data_info in datainfo_list:
    df = df_index_stat(data_info)
    df = df[df.theta==theta]
    for setting in setting_list:
        try: t = "{:.3f}".format(df[df.setting==setting]['Time_BuildIndex'].values[0])
        except: t = '-'
        try: s = "{:,}".format(df[df.setting==setting]['Space_Index'].values[0])
        except: s = '-'
        print(' & '.join([str(data_info), setting, t, s]) +' \\\\\\hline')

In [ ]:
def table_index_size_vary_dataset(alg):
    dataset_list = ['WIKI', 'AMAZON', 'PUBMED']
    SEP = ' '
    
    def get_df(datainfo):
        df = df_time_by_size(datainfo, alg_list=[alg])
        if alg.startswith('PrefixSearch'):
            return df[df.theta==0.7][df['setting'].isin(['RSS-P'])][['n', 'Space_Index']]
        else:
            return df[df.theta==0.7][['n', 'Space_Index']]
    
    with open('data_size.txt') as f:
        lines = list(map(str.split, f.readlines()))
    dict_data_size = {(x[0], x[1]):x[2] for x in lines}
    dict_datainfo = {
        'WIKI':DataInfo('WIKI', '*', '107836', '5', '1.0', '10'),
        'AMAZON':DataInfo('AMAZON', '*', '107836', '5', '1.0', '10'),
        'PUBMED':DataInfo('PUBMED', '*', '79011', '5', '1.0', '10'),
    }
    dict_df = {x:get_df(dict_datainfo[x]) for x in dataset_list}
    
    ratio_mat = []
    for n in ['10000', '31622', '100000', '316227', '1000000']:
        val_list = []
        ratio_list = []
        for dataset in ['WIKI', 'AMAZON', 'PUBMED']:
            df = dict_df[dataset]
            size_d = float(dict_data_size[(dataset, n)])/1e6
            val_list.append(f"{size_d:.3f}")
            try: 
                size_i = float(df[df.n==int(n)]['Space_Index'].item())
                ratio_list.append(size_i/size_d)
                val_list.append(f"{size_i:.3f}")
            except:
                size_i = '-'
                ratio_list.append(0)
                val_list.append(size_i)
        ratio_mat.append(ratio_list)
            
        print(f"$10^{{{np.log10(int(n)):.1f}}}$"+SEP+SEP.join(val_list)+' \\\\\\hline')
    
    ratio_arr = np.array(ratio_mat)
    print(np.mean(ratio_arr, axis=0))
# table_index_size_vary_dataset("PrefixSearch_6.32")
# table_index_size_vary_dataset("PkwiseSynSearch_2.00")
# table_index_size_vary_dataset("FaerieSynSearch_1.02")

In [ ]:
def table_index_size_vary_alg(datainfo, alg_list):
    SEP = ' & '
    
    def get_df(alg):
        df = df_time_by_size(datainfo, alg_list=[alg])
        if len(df) == 0: return None
        if alg.startswith('PrefixSearch'):
            return df[df.theta==0.7][df['setting'].isin(['RSS-CP'])][['n', 'Space_Index']]
        else:
            return df[df.theta==0.7][['n', 'Space_Index']]
    
    with open('data_size.txt') as f:
        lines = list(map(str.split, f.readlines()))
    dict_data_size = {(x[0], x[1]):x[2] for x in lines}

    dict_df = {x:get_df(x) for x in alg_list}
    
    ratio_mat = []
    for n in ['10000', '31622', '100000', '316227', '1000000', '3162277', '10000000']:
        val_list = []
        ratio_list = []
        size_d = float(dict_data_size[(datainfo.name, n)])/1e6
        val_list.append(f"{size_d:.3f}")
        for alg in alg_list:
            df = dict_df[alg]
            try: 
                size_i = float(df[df.n==int(n)]['Space_Index'].item())
                ratio_list.append(size_i/size_d)
                val_list.append(f"{size_i:.3f}")
            except:
                size_i = '-'
                ratio_list.append(0)
                val_list.append(size_i)
        ratio_mat.append(ratio_list)
            
        print(f"$10^{{{np.log10(int(n)):.1f}}}$"+SEP+SEP.join(val_list)+' \\\\\\hline')
    
    ratio_arr = np.array(ratio_mat)
    print(np.mean(ratio_arr, axis=0))
# datainfo = DataInfo('WIKI', '*', '107836', '5', '1.0', NAR)
# table_index_size_vary_alg(datainfo, ["PrefixSearch_6.32", "PkwiseSynSearch_2.00", "FaerieSynSearch_1.02"])

In [ ]:
def compare_filter_verify():
    dataset_list = ['WIKI', 'AMAZON', 'PUBMED']
    dict_datainfo = {
        'WIKI':DataInfo('WIKI', '*', '107836', '5', '1.0', '-1'),
        'AMAZON':DataInfo('AMAZON', '*', '107836', '5', '1.0', '-1'),
        'PUBMED':DataInfo('PUBMED', '*', '79011', '5', '1.0', '-1'),
    }
    SEP = ' & '

    def get_df(datainfo):
        df = df_time_by_size(datainfo, alg_list=["PrefixSearch_6.32"])
        return  df[df.theta==0.7][df['setting'].isin(['RSS-CPL'])][['n', 'nq', 'Time_IndexFilter', 'Time_Validation']]

    with open('data_size.txt') as f:
        lines = list(map(str.split, f.readlines()))
    dict_data_size = {(x[0], x[1]):x[2] for x in lines}
    dict_datainfo = {
        'WIKI':DataInfo('WIKI', '*', '107836', '5', '1.0', '-1'),
        'AMAZON':DataInfo('AMAZON', '*', '107836', '5', '1.0', '-1'),
        'PUBMED':DataInfo('PUBMED', '*', '79011', '5', '1.0', '-1'),
    }
    dict_df = {x:get_df(dict_datainfo[x]) for x in dataset_list}
    
    ratio_mat = []
    for n in ['10000', '31622', '100000', '316227', '1000000']:
        val_list = []
        ratio_list = []
        for dataset in ['WIKI', 'AMAZON', 'PUBMED']:
            df = dict_df[dataset]
            t_f = float(df[df.n==int(n)]['Time_IndexFilter'].item())/int(df[df.n==int(n)]['nq'])*1000 # ms
            t_v = float(df[df.n==int(n)]['Time_Validation'].item())/int(df[df.n==int(n)]['nq'])*1000 # ms
            ratio_list.append(t_f/t_v)
            val_list += [t_f, t_v]
        ratio_mat.append(ratio_list)
            
        print(f"$10^{{{np.log10(int(n)):.1f}}}$"+SEP+SEP.join(map(lambda x: f"{x:.3f}", val_list))+' \\\\\\hline')
    
    print(pd.DataFrame(ratio_mat))
    print(pd.DataFrame(ratio_mat).min(axis=0))
# compare_filter_verify()